In [1]:
import os
import cv2
import random
import numpy as np
import pandas as pd
from glob import glob
from sklearn.utils import shuffle

In [2]:
fake_paths=glob('d:/AI_term_project/face_crop/fake/*.jpg')
real_paths=glob('d:/AI_term_project/face_crop/real1/*.jpg')

print(len(fake_paths))
print(len(real_paths))

all_paths=fake_paths+real_paths

169868
134967


In [3]:
# Face Mask
import cv2
from imutils import face_utils
import dlib
detector = dlib.get_frontal_face_detector ()
predictor = dlib.shape_predictor('d:/AI_term_project/shape_predictor_68_face_landmarks.dat')

def blind_face(img):
    seed = random.random ()
    probability = 0.5

    if (seed < probability):
        seed = random.random ()
        case = int (seed * 3)
        gray = cv2.cvtColor (img, cv2.COLOR_RGB2GRAY)

        results = detector(gray, 0)
        if len(results) > 0 :
            shape = predictor (gray, results[0])
            shape = face_utils.shape_to_np (shape)
            pts_ = get_poly (shape, case, img.shape[1], img.shape[0])
            cv2.fillConvexPoly (img, pts_, (0, 0, 0))
            del shape, pts_

        
def get_poly(points, case_, x, y):
    input_shape = (x, y)

    pts = np.array ([points[36], points[20], points[23], points[45]], np.int32)

    if case_ == 0:  # both eye
        pts = np.array ([[points[36][0]-10, points[36][1]-20], [points[36][0]-10, points[36][1]+20], [points[45][0]+10, points[45][1]+20], [points[45][0]+10, points[45][1]-20]], np.int32)
    elif case_ == 1:  # nose
        pt1 = np.array ([points[31][0], points[28][1]])
        pt2 = np.array ([points[35][0], points[28][1]])
        pts = np.array ([points[27], pt1, points[31], points[51], points[35], pt2], np.int32)
    elif case_ == 2:  # forehead
        pt1 = np.array ([0, points[17][1]])
        pt2 = np.array ([input_shape[0] - 1, points[26][1]])
        pt3 = np.array ([input_shape[0] - 1, 0])
        pt4 = np.array ([0, 0])
        pts = np.array ([pt1, pt2, pt3, pt4], np.int32)
    
    return pts

In [5]:
from tensorflow.keras.utils import Sequence
import albumentations as A
import cv2

labeling={'fake':1, 'real':0, 'test':2}

def load_img_label(path):
    image = tf.io.read_file(path)
    image = tf.image.decode_jpeg(image, channels=3)
    label = labeling[path.split('/')[-1][:4]]
    return image, label

class DataGenerator(Sequence):

    def __init__(self, X, y, batch_size, step_per_epoch, input_shape, shuffle=True, augment = True):
        self.X = X
        self.y = y if y is not None else None
        self.batch_size = batch_size
        self.step_per_epoch = step_per_epoch
        self.input_shape = input_shape
        self.shuffle = shuffle
        self.augment = augment
        self.augment_size = int(self.batch_size*0.3)
        self.init_generator()
        self.on_epoch_end()

    def _shuffle_sample(self):
        if (self.shuffle == True):
            np.random.shuffle(self.indexes)

    def init_generator(self):
        self.sample_size = len(self.X)
        self.indexes = np.zeros(self.sample_size, dtype=np.int)

        for i in range(self.sample_size):
            self.indexes[i] = int(i)

        self._shuffle_sample()

        self.miniEpoch_size = self.batch_size * self.step_per_epoch
        self.cnt_mini_epoch = self.sample_size // self.miniEpoch_size
        self.iter_mini_epoch = 0

    def on_epoch_end(self):
        if (self.iter_mini_epoch < self.cnt_mini_epoch):
            self.mini_indexes = self.indexes[self.iter_mini_epoch * self.miniEpoch_size: (self.iter_mini_epoch + 1) * self.miniEpoch_size]
            self.iter_mini_epoch = self.iter_mini_epoch + 1
        else:
            self.iter_mini_epoch = 0
            self._shuffle_sample()
            self.on_epoch_end()

    def __len__(self):
        return int(self.step_per_epoch)

    def __data_generation(self, X_list, y_list):
        X = np.zeros ((self.batch_size, self.input_shape[0], self.input_shape[1], self.input_shape[2]), dtype=np.float32)
        y = np.zeros ((self.batch_size,), dtype=int)

        if y_list is not None:
            for i, (path, label) in enumerate(zip(X_list, y_list)):
                img, label = load_img_label(path)
                if self.augment and i < self.augment_size:
                    img = augmentor(img)
                X[i] = tf.image.resize(tf.cast(img, tf.float32), (380,380)) / 255.0
                y[i] = label

            return X, y
        else:
            for i, path in enumerate (X_list):
                img, _ = load_img_label(path)
                X[i] = tf.image.resize(tf.cast(img, tf.float32), (380,380)) / 255.0

            return X

    def __getitem__(self, index):
        indexes = self.mini_indexes[index * self.batch_size: (index + 1) * self.batch_size]
        X_list = [self.X[k] for k in indexes]

        if self.y is not None:
            y_list = [self.y[k] for k in indexes]
            X, y = self.__data_generation (X_list, y_list)
            return X, y
        else:
            y_list = None
            X = self.__data_generation (X_list, y_list)
            return X

def augmentor(img):
    image = np.array(img)
    blind_face(image)
    transform = A.Compose([
            A.augmentations.transforms.HorizontalFlip(p=0.5),
            A.augmentations.transforms.GaussianBlur(p=0.3),
            A.augmentations.transforms.GaussNoise(p=0.3),
            A.augmentations.transforms.ImageCompression(quality_lower=60, quality_upper=100, p=0.3),
    ])
    transformed = transform(image=image)
    image = transformed["image"]
    return image
        

In [6]:
train_data = DataGenerator(all_paths, all_paths, 6, 50805, (380, 380, 3), True, True)

C:\Users\sclab\AppData\Local\Temp\ipykernel_5100\3494557681.py:33: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.indexes = np.zeros(self.sample_size, dtype=np.int)


In [7]:
import tensorflow as tf
from tensorflow import keras
import efficientnet.tfkeras as efficientnet
import tensorflow_addons as tfa

def model():
    pretrained_model = efficientnet.EfficientNetB4(
        include_top=False,
        weights='imagenet',
        input_shape=(380, 380, 3),
      )
    pretrained_model.trainable = True
  
    inputs = keras.layers.Input(shape=(380, 380, 3))
    x = pretrained_model(inputs)
    x = keras.layers.GlobalAveragePooling2D()(x)
    x = keras.layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01))(x)
    x = keras.layers.Dropout(0.5)(x)
    output = keras.layers.Dense(1, activation='sigmoid')(x)
    model = keras.models.Model(inputs = inputs, outputs = output)

    model.compile(
        optimizer = tfa.optimizers.RectifiedAdam(lr=1e-5, weight_decay=0.0005), 
        loss=keras.losses.BinaryCrossentropy(from_logits=False),
        metrics = ['accuracy']
    )

    return model

model = model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 380, 380, 3)]     0         
                                                                 
 efficientnet-b4 (Functional  (None, 12, 12, 1792)     17673816  
 )                                                               
                                                                 
 global_average_pooling2d (G  (None, 1792)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 512)               918016    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 513   

C:\ProgramData\Anaconda3\envs\test_v2\lib\site-packages\tensorflow_addons\optimizers\rectified_adam.py:120: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [61]:
model.load_weights('d:/AI_term_project/model1.hdf5')

In [8]:
from keras import callbacks

filepath = 'd:/AI_term_project/380380.hdf5'

mcp = callbacks.ModelCheckpoint(filepath, monitor = 'loss', save_best_only=False, save_weights_only=True, mode='min', save_freq=50, verbose=0)
lrp = callbacks.ReduceLROnPlateau(monitor = 'loss', factor=0.25, patience=2, mode='min')

In [ ]:
model.fit(train_data, epochs= 5, verbose=1, callbacks=[mcp])

Epoch 1/5
   40/50805 [..............................] - ETA: 6:52:01 - loss: 8.6820 - accuracy: 0.4917

In [64]:
test_path = 'd:/AI_term_project/test/test_crop/*.jpg'

# collect all images
images = glob(test_path)
images.sort()
print(len(images))
test_data = DataGenerator(images, None, 1, 4100, (224, 224, 3), False, False)

4100


C:\Users\sclab\AppData\Local\Temp\ipykernel_18388\173843326.py:33: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.indexes = np.zeros(self.sample_size, dtype=np.int)


In [69]:
model.load_weights('d:/AI_term_project/rectifiedAdam_90.5%.hdf5')
p = model.predict(test_data)
model.load_weights('d:/AI_term_project/89%.hdf5')
p2 = model.predict(test_data)

In [70]:
dir = 'd:/AI_term_project/submission.csv'
sc = open(dir, 'w')
sc.write('path,y')
sc.write('\n')

for i in range(len(images)):
    if p[i]+p2[i] > 1.0:
        saveline = 'leaderboard/'+images[i][-15:] + ',1'
        sc.write(saveline)
        sc.write('\n')
    else:
        saveline = 'leaderboard/'+images[i][-15:] + ',0'
        sc.write(saveline)
        sc.write('\n')

sc.close()

submission = pd.read_csv(dir, index_col = False)
print(submission["y"].value_counts())

0    2117
1    1983
Name: y, dtype: int64
